In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DataCleaning") \
    .master("local[*]") \
    .getOrCreate()

df = spark.read.csv("work/ecommerce_transactions_1000.csv", header=True, inferSchema=True)
df.show(5)
print("jumlah baris:", df.count())

+--------------+-------+--------+--------------------+-------------------+
|transaction_id|user_id|  amount|               email|   transaction_time|
+--------------+-------+--------+--------------------+-------------------+
|         T0001|   U069|    NULL|jeffreyfisher@gma...|2025-04-20 08:00:02|
|         T0002|   U253|70921.08| porteramy@yahoo.com|2025-03-30 21:07:41|
|         T0003|   U222|42313.74|  jerome93@yahoo.com|2025-04-20 10:50:30|
|         T0004|   U187|    NULL|jimeneztamara@sny...|2025-04-05 11:48:29|
|         T0005|   U064|81176.73|   louis64@gmail.com|2025-04-14 08:50:35|
+--------------+-------+--------+--------------------+-------------------+
only showing top 5 rows

jumlah baris: 1000


In [5]:
from pyspark.sql.functions import col, when, count
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+--------------+-------+------+-----+----------------+
|transaction_id|user_id|amount|email|transaction_time|
+--------------+-------+------+-----+----------------+
|             0|      0|   316|    0|              50|
+--------------+-------+------+-----+----------------+



In [7]:
print("jumlah baris:", df.count())

jumlah baris: 1000


In [11]:
df = df.dropna(subset=["transaction_time"])
df = df.fillna({"amount": 0})

In [13]:
from pyspark.sql.functions import instr, substring_index, col

# Tambah kolom email_domain
df = df.withColumn("email_domain", substring_index("email", "@", -1))

# Filter hanya email yang mengandung '@'
df = df.filter(instr(col("email"), "@") > 0)

In [14]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import to_date, col

# Konversi kolom 'amount' ke DoubleType
df = df.withColumn("amount", col("amount").cast(DoubleType()))

# Konversi 'transaction_time' ke tanggal, dengan format eksplisit jika diperlukan
df = df.withColumn("transaction_date", to_date(col("transaction_time"), "yyyy-MM-dd"))


In [15]:
df.write.csv("cleaned_transactions_1000.csv", header=True, mode="overwrite")


In [16]:
df.filter(col("transaction_time").isNull()).count()


0

In [ ]:
 Checklist Praktikum 
●	Dataset berhasil dibaca 
●	Struktur schema dipahami 
●	Missing value berhasil dihandle 
●	Email valid berhasil diproses 
●	Kolom baru  email_domain  dan  transaction_date  berhasil  dibuat 
●	Data bersih disimpan ulang 
 Pertanyaan 
1.	Berapa banyak data yang dibuang karena  transaction_time  kosong?
jawab:
    jumlah di hitung menggunakan 
    df.filter(col("transaction_time").isNull()).count()

2.	Apakah semua data  amount  sudah bertipe numerik setelah cleaning? 
jawab:
    Ya, jika sudah di-cast ke DoubleType dan tidak ada null.

3.	Kenapa lebih baik memperbaiki email invalid sebelum menganalisis data  transaksi?
jawab:
    Karena email invalid bisa membuat analisis salah dan tidak bisa digunakan
    untuk identifikasi atau pengelompokan.


In [ ]:
 Praktikum 2: Deteksi Outlier Sederhana di Spark 

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

# Inisialisasi SparkSession
spark = SparkSession.builder.appName("OutlierDetection").getOrCreate()

# Baca file CSV
df = spark.read.csv("work/ecommerce_transactions_1000.csv", header=True, inferSchema=True)

# Ubah kolom amount jadi tipe double
df = df.withColumn("amount", col("amount").cast("double"))

# Tambahkan kolom transaction_date dari transaction_time
df = df.withColumn("transaction_date", to_date("transaction_time"))


In [22]:
# Hitung Q1 (25%) dan Q3 (75%)
quantiles = df.approxQuantile("amount", [0.25, 0.75], 0.01)
Q1 = quantiles[0]
Q3 = quantiles[1]

# Hitung IQR
IQR = Q3 - Q1

# Tentukan batas bawah dan atas untuk outlier
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter outlier
outliers = df.filter((df["amount"] < lower_bound) | (df["amount"] > upper_bound))

# Cetak jumlah outlier
print("Jumlah Outliers:", outliers.count())


Jumlah Outliers: 331


In [23]:
# Filter baris yang merupakan outlier
outliers = df.filter((df["amount"] < lower_bound) | (df["amount"] > upper_bound))

# Menampilkan jumlah outliers
print("Jumlah Outliers:", outliers.count())

# (Opsional) Menampilkan beberapa data outliers
outliers.show(5)


Jumlah Outliers: 331
+--------------+-------+------+--------------------+-------------------+----------------+
|transaction_id|user_id|amount|               email|   transaction_time|transaction_date|
+--------------+-------+------+--------------------+-------------------+----------------+
|         T0008|   U212|   NaN|  dgreen@hotmail.com|2025-04-23 07:19:12|      2025-04-23|
|         T0010|   U033|   NaN|rebecca69@hotmail...|2025-04-15 04:04:31|      2025-04-15|
|         T0013|   U184|   NaN|jackielewis@yahoo...|2025-03-29 21:00:47|      2025-03-29|
|         T0014|   U130|   NaN|    dawn56@roman.net|2025-04-15 19:21:50|      2025-04-15|
|         T0019|   U280|   NaN|   hgarcia@yahoo.com|2025-04-12 00:43:15|      2025-04-12|
+--------------+-------+------+--------------------+-------------------+----------------+
only showing top 5 rows



In [25]:
quantiles = df.approxQuantile("amount", [0.25, 0.75], 0.05)
Q1, Q3 = quantiles
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Q1 = {Q1}, Q3 = {Q3}, IQR = {IQR}")
print(f"Lower Bound = {lower_bound}, Upper Bound = {upper_bound}")

outliers = df.filter((df["amount"] < lower_bound) | (df["amount"] > upper_bound))
print("Jumlah Outliers:", outliers.count())


Q1 = 34005.04, Q3 = 74468.55, IQR = 40463.51
Lower Bound = -26690.225, Upper Bound = 135163.815
Jumlah Outliers: 331


In [26]:
outliers = df.filter((df.amount < lower_bound) | (df.amount > upper_bound))
outliers.show()


+--------------+-------+------+--------------------+-------------------+----------------+
|transaction_id|user_id|amount|               email|   transaction_time|transaction_date|
+--------------+-------+------+--------------------+-------------------+----------------+
|         T0008|   U212|   NaN|  dgreen@hotmail.com|2025-04-23 07:19:12|      2025-04-23|
|         T0010|   U033|   NaN|rebecca69@hotmail...|2025-04-15 04:04:31|      2025-04-15|
|         T0013|   U184|   NaN|jackielewis@yahoo...|2025-03-29 21:00:47|      2025-03-29|
|         T0014|   U130|   NaN|    dawn56@roman.net|2025-04-15 19:21:50|      2025-04-15|
|         T0019|   U280|   NaN|   hgarcia@yahoo.com|2025-04-12 00:43:15|      2025-04-12|
|         T0020|   U057|   NaN|    paul68@yahoo.com|2025-04-15 11:48:24|      2025-04-15|
|         T0022|   U157|   NaN|    ysilva@gmail.com|2025-04-05 14:14:18|      2025-04-05|
|         T0023|   U085|   NaN|   shawn41@yahoo.com|2025-04-26 23:15:02|      2025-04-26|
|         

In [27]:
df.orderBy(df.amount.desc()).show(5)


+--------------+-------+------+--------------------+-------------------+----------------+
|transaction_id|user_id|amount|               email|   transaction_time|transaction_date|
+--------------+-------+------+--------------------+-------------------+----------------+
|         T0019|   U280|   NaN|   hgarcia@yahoo.com|2025-04-12 00:43:15|      2025-04-12|
|         T0028|   U110|   NaN|elizabethmclean@p...|2025-04-26 14:43:19|      2025-04-26|
|         T0020|   U057|   NaN|    paul68@yahoo.com|2025-04-15 11:48:24|      2025-04-15|
|         T0014|   U130|   NaN|    dawn56@roman.net|2025-04-15 19:21:50|      2025-04-15|
|         T0022|   U157|   NaN|    ysilva@gmail.com|2025-04-05 14:14:18|      2025-04-05|
+--------------+-------+------+--------------------+-------------------+----------------+
only showing top 5 rows



In [ ]:
Penjelasan:
Perintah ini mengurutkan data berdasarkan kolom amount secara menurun (desc()), lalu menampilkan 5 baris teratas. Ini berguna untuk mengetahui transaksi dengan nilai paling besar dalam dataset.

In [28]:
total_transaksi = df.count()
print("Jumlah total transaksi:", total_transaksi)


Jumlah total transaksi: 1000


In [ ]:
Penjelasan:
Fungsi count() menghitung jumlah baris dalam DataFrame, yang mewakili jumlah total transaksi. Ini membantu kita mengetahui skala data yang sedang dianalisis.

In [29]:
jumlah_outlier = outliers.count()
print("Jumlah outlier:", jumlah_outlier)


Jumlah outlier: 331


In [30]:
persentase_outlier = (jumlah_outlier / total_transaksi) * 100
print(f"Persentase outlier: {persentase_outlier:.2f}%")


Persentase outlier: 33.10%
